# Linear Regression Via the method of Least Squares #

# How this code fits the Tripp relation with matrix least squares

## Goal
$\mu_{\text{obs}} = -2.5\log_{10}x_0 + \alpha x_1 - \beta c + M_B,$  

$\mu_{\text{th}} = [-2.5\log_{10}x_0] + \alpha x_1 - \beta c + M_B.$

$\mu_0 \equiv -2.5\log_{10}x_0,\quad y \equiv \mu_{\text{th}} - \mu_0.$  

$y \approx \alpha x_1 - \beta c + M_B.$

## Matrix setup
The following is called a predictor matrix (i.e. our data matrix from ZTF):

$
X =
\begin{bmatrix}
x_{1,1} & -c_1 & 1 \\
x_{1,2} & -c_2 & 1 \\
\vdots  & \vdots & \vdots \\
x_{1,N} & -c_N & 1
\end{bmatrix}.
$

- **Column 1** = $x_1$ (slope for $\alpha$)  
- **Column 2** = $-c$ (slope for $\beta$; the minus sign bakes in the negative so $\theta_2$ directly corresponds to $\beta$)  
- **Column 3** = $1$ (intercept column, representing $M_B$)

Stack the targets into a vector $y \in \mathbb{R}^{N}$:  

$
y =
\begin{bmatrix}
\mu_{\text{th},1} - \mu_{0,1} \\
\mu_{\text{th},2} - \mu_{0,2} \\
\vdots \\
\mu_{\text{th},N} - \mu_{0,N}
\end{bmatrix}.
$

Collect the unknowns into $\theta \in \mathbb{R}^3$:  

$
\theta =
\begin{bmatrix}
\alpha \\
\beta \\
M_B
\end{bmatrix}.
$

If you setup the matrices in this manner:

$\hat{y} = X\theta$

and mulitply these out. Then you will be able to recover your original equation:

$ \alpha x_1 + \beta(-c) + M_B\cdot1.$

You Plug this into a least squares function and it gives you back the theta matrix that has all your relevant parameters. 

---

## What the solver minimizes
The code computes the **ordinary least-squares (OLS)** solution:  
$\theta^\star = \arg\min_{\theta} \|y - X\theta\|_2^2 = \arg\min_{\theta} (y - X\theta)^\top (y - X\theta).$

Setting the gradient to zero gives the **normal equations**:  
$X^\top X\,\theta^\star = X^\top y.$

If $X^\top X$ is invertible, the closed-form solution is  
$\boxed{\theta^\star = (X^\top X)^{-1} X^\top y}$  
which is exactly what `np.linalg.lstsq` computes (with numerical safeguards).

After solving, the code unpacks  
$\alpha = \theta^\star_1,\quad \beta = \theta^\star_2,\quad M_B = \theta^\star_3.$


In [1]:
import numpy as np
import pandas as pd
from astropy.cosmology import FlatLambdaCDM

# --- Load the ZTF data ---
csv_path = "/Users/pittsburghgraduatestudent/repos/myc21_ztf_mu/ZTF_snia_data.csv"
df = pd.read_csv(csv_path)

# Drop rows missing SALT2 parameters
df_clean = df.dropna(subset=["x0", "x1", "c", "redshift"]).copy()

# --- Extract arrays ---
x0 = df_clean["x0"].to_numpy()
x1 = df_clean["x1"].to_numpy()
c  = df_clean["c"].to_numpy()
z  = df_clean["redshift"].to_numpy()

# --- Cosmology & theoretical distance modulus ---
H0, OM0 = 70.0, 0.3
cosmo = FlatLambdaCDM(H0=H0, Om0=OM0)
mu_th = cosmo.distmod(z).value

# --- Convert amplitude to magnitude-like term ---
mu0 = -2.5 * np.log10(x0)

# --- Define regression target ---
# Tripp: mu_th = mu0 + alpha*x1 - beta*c + MB
y = mu_th - mu0
X = np.column_stack([x1, -c, np.ones_like(x1)])  # [x1, -c, intercept]

# --- Solve least squares ---
theta, *_ = np.linalg.lstsq(X, y, rcond=None)
alpha, beta, MB = theta

# --- Residuals and naive covariance ---
r   = y - X @ theta
dof = max(len(r) - 3, 1)
s2  = float(r @ r / dof)
cov = s2 * np.linalg.inv(X.T @ X)
se_alpha, se_beta, se_MB = np.sqrt(np.diag(cov))

# --- Print results ---
print(f"alpha = {alpha:.3f} ± {se_alpha:.3f}")
print(f"beta  = {beta:.3f} ± {se_beta:.3f}")
print(f"MB    = {MB:.3f} ± {se_MB:.3f}")

alpha = 0.120 ± 0.005
beta  = 2.605 ± 0.034
MB    = 29.715 ± 0.007



$
\chi^2 = \sum_i \frac{(\mu_{\text{obs},i} - \mu_{\text{th},i})^2}
               {\sigma_{\mu,i}^2}
\quad \text{where} \quad
\sigma_{\mu,i}^2 = \sigma_{\mu,\text{meas},i}^2 + \sigma_{\text{int}}^2 + \cdots
$

$
\mu_{\text{obs}} = -2.5 \log_{10}(x_0) + \alpha x_1 - \beta c + M_B
$

where:
- $x_0$ = amplitude from the SALT2 light-curve fit  
- $x_1$ = stretch parameter  
- $c$ = color parameter  
- $\alpha, \beta, M_B$ are global nuisance parameters.

Partial Derviatives that end up helping out later:

with partial derivatives:
$
\frac{\partial \mu}{\partial x_0} = -\frac{2.5}{x_0 \ln 10}, \quad
\frac{\partial \mu}{\partial x_1} = +\alpha, \quad
\frac{\partial \mu}{\partial c}   = -\beta
$


General Form of the Error Propagation Formula for $y = f(x_1, x_2, \ldots, x_n)$

$$
\sigma_y^2 =
\sum_{i=1}^{n}
\left(
\frac{\partial f}{\partial x_i}
\right)^2
\sigma_{x_i}^2
+
\sum_{i \ne j}
2\,
\frac{\partial f}{\partial x_i}
\frac{\partial f}{\partial x_j}
\mathrm{Cov}(x_i, x_j).
$$

Using the standard error propagation formula:

$$
\sigma_{\mu,\text{meas}}^2 =
\left(\frac{\partial \mu}{\partial x_0}\right)^2 \sigma_{x_0}^2
+\left(\frac{\partial \mu}{\partial x_1}\right)^2 \sigma_{x_1}^2
+\left(\frac{\partial \mu}{\partial c}\right)^2 \sigma_c^2
+ 2\left(\frac{\partial \mu}{\partial x_0}\frac{\partial \mu}{\partial x_1}\right)\mathrm{Cov}(x_0,x_1)
+ 2\left(\frac{\partial \mu}{\partial x_0}\frac{\partial \mu}{\partial c}\right)\mathrm{Cov}(x_0,c)
+ 2\left(\frac{\partial \mu}{\partial x_1}\frac{\partial \mu}{\partial c}\right)\mathrm{Cov}(x_1,c)
$$


In [3]:
import numpy as np
import pandas as pd

# =========================
# Helper functions
# =========================
LN10 = np.log(10.0)

def sigma_mu_meas2(alpha, beta, x0, x1, c, sig_x0, sig_x1, sig_c, cov_x0_x1, cov_x0_c, cov_x1_c):
    """
    Measurement variance of mu_obs given the parameters given in the ZTF SNIa release.
    """

    dmu_dx0 = -2.5 / (x0 * LN10)
    dmu_dx1 = +alpha
    dmu_dc  = -beta

    var = (dmu_dx0**2) * (sig_x0**2) \
        + (dmu_dx1**2) * (sig_x1**2) \
        + (dmu_dc**2)  * (sig_c**2)

    var += 2.0 * (dmu_dx0 * dmu_dx1) * cov_x0_x1
    var += 2.0 * (dmu_dx0 * dmu_dc)  * cov_x0_c
    var += 2.0 * (dmu_dx1 * dmu_dc)  * cov_x1_c
    return var

### Weighted Linear Least-Squares Fit

The function performs a **weighted linear least-squares fit** for a model of the form  

$$
y = X\,\theta + \epsilon,
$$

where  

- $y$: vector of observations (size $N \times 1$)  
- $X$: **design matrix** (size $N \times p$) containing predictors (e.g. $x_1$, $-c$, intercept, etc.)  
- $\theta$: vector of fit parameters (e.g. $[\alpha, \beta, M_B]$)  
- $w$: weights (usually $w_i = 1 / \sigma_i^2$)

The best-fit parameters $\hat{\theta}$ minimize the weighted sum of squared residuals:  

$$
\chi^2 = \sum_i w_i \, \big(y_i - (X\theta)_i\big)^2.
$$

In [ ]:
def wls_theta(X, y, w):
    """Weighted least squares (diagonal weights)."""
    WX = X * w[:, None]
    XtWX = X.T @ WX
    XtWy = X.T @ (w * y)
    theta = np.linalg.solve(XtWX, XtWy)
    cov_theta = np.linalg.inv(XtWX)
    return theta, cov_theta

N used     = 3576
alpha      = 0.1366676372986602
beta       = 2.9714088034454895
MB         = 29.7351252588439
sigma_int  = 0.3081265926361083
chi2_red   = 1.014236120585226
Cov(alpha,beta,MB)=
 [[ 2.04086677e-05 -1.73215934e-05 -1.84352375e-07]
 [-1.73215934e-05  1.00799943e-03  4.17091465e-05]
 [-1.84352375e-07  4.17091465e-05  3.22256630e-05]]


In [ ]:
def mle_update_sigma_int(resid, var_no_int, tol=1e-8, max_iter=100):
    """
    MLE update for intrinsic scatter sigma_int given:
        var_tot = var_no_int + sigma_int^2
    Solve sum(1/var_tot) = sum(resid^2/var_tot^2) via bisection.
    """
    def F(s):
        v = var_no_int + s*s
        lhs = np.sum(1.0 / v)
        rhs = np.sum((resid**2) / (v**2))
        return lhs - rhs

    s_lo, f_lo = 0.0, F(0.0)
    s_hi = 0.3
    f_hi = F(s_hi)
    tries = 0
    while f_lo * f_hi > 0.0 and s_hi < 1.0 and tries < 20:
        s_hi *= 1.5
        f_hi = F(s_hi)
        tries += 1
    if f_lo * f_hi > 0.0: 
        return max(0.0, s_lo if abs(f_lo) < abs(f_hi) else s_hi)

    for _ in range(max_iter):
        s_mid = 0.5 * (s_lo + s_hi)
        f_mid = F(s_mid)
        if abs(f_mid) < 1e-12 or (s_hi - s_lo) < tol:
            return max(0.0, s_mid)
        if f_lo * f_mid <= 0.0:
            s_hi, f_hi = s_mid, f_mid
        else:
            s_lo, f_lo = s_mid, f_mid
    return max(0.0, 0.5 * (s_lo + s_hi))


def fit_salt2_nuisance(
    z, mu_th,
    x0, x1, c,
    sig_x0, sig_x1, sig_c,
    cov_x0_x1=None, cov_x0_c=None, cov_x1_c=None,
    init_alpha=0.14, init_beta=3.1, init_MB=-19.3,
    init_sigma_int=0.10,
    max_outer_iter=50, tol_theta=1e-8, tol_sigint=1e-6,
    update_sigma_int_mode="MLE"
):
    """
    Fit (alpha, beta, M_B) with iterative reweighting and MLE update of sigma_int,
    using ONLY measurement propagation + intrinsic scatter (NO peculiar velocity term).
    """
    N = len(x0)
    if cov_x0_x1 is None: cov_x0_x1 = np.zeros(N)
    if cov_x0_c  is None: cov_x0_c  = np.zeros(N)
    if cov_x1_c  is None: cov_x1_c  = np.zeros(N)

    alpha, beta, MB = float(init_alpha), float(init_beta), float(init_MB)
    sigma_int = float(init_sigma_int)

    for _ in range(max_outer_iter):
        # Build regression: y ≈ X @ [alpha, beta, MB]
        y = mu_th + 2.5 * np.log10(x0)
        X = np.column_stack([x1, -c, np.ones_like(x1)])

        # Per-SN measurement variance (depends on alpha,beta)
        var_meas = sigma_mu_meas2(alpha, beta, x0, x1, c,
                                  sig_x0, sig_x1, sig_c,
                                  cov_x0_x1, cov_x0_c, cov_x1_c)

        # Total variance = measurement + intrinsic^2   (NO peculiar velocity)
        var_tot = var_meas + (sigma_int**2)
        w = 1.0 / var_tot

        # Weighted least squares
        theta, cov_theta = wls_theta(X, y, w)
        alpha_new, beta_new, MB_new = theta.tolist()

        # Residuals
        r = y - X @ theta

        # Update sigma_int
        if update_sigma_int_mode.upper() == "MLE":
            var_no_int = var_meas
            sigma_int_new = mle_update_sigma_int(r, var_no_int, tol=tol_sigint)
        else:
            dof = max(1, N - 3)
            chi2 = float((r**2 * w).sum())
            target = chi2 / dof
            sigma_int_new = max(0.0, sigma_int * np.sqrt(target))

        # Convergence
        dtheta = np.max(np.abs([alpha_new - alpha, beta_new - beta, MB_new - MB]))
        ds = abs(sigma_int_new - sigma_int)
        alpha, beta, MB, sigma_int = alpha_new, beta_new, MB_new, sigma_int_new
        if dtheta < tol_theta and ds < tol_sigint:
            break

    # Final pass for outputs
    y = mu_th + 2.5 * np.log10(x0)
    X = np.column_stack([x1, -c, np.ones_like(x1)])
    var_meas = sigma_mu_meas2(alpha, beta, x0, x1, c,
                              sig_x0, sig_x1, sig_c,
                              cov_x0_x1, cov_x0_c, cov_x1_c)
    var_tot = var_meas + (sigma_int**2)
    w = 1.0 / var_tot
    theta, cov_theta = wls_theta(X, y, w)
    r = y - X @ theta
    chi2 = float((r**2 * w).sum())
    dof = max(1, len(y) - 3)
    chi2_red = chi2 / dof

    return dict(
        alpha=alpha, beta=beta, MB=MB,
        cov=cov_theta,
        sigma_int=sigma_int,
        chi2=chi2, dof=dof, chi2_reduced=chi2_red,
        residuals=r, weights=w
    )

# =========================
# Load your ZTF CSV & run
# =========================
csv_path = "/Users/pittsburghgraduatestudent/repos/myc21_ztf_mu/ZTF_snia_data.csv"
df = pd.read_csv(csv_path)

# Keep rows with required SALT2 params
need = ["redshift", "x0", "x1", "c", "x0_err", "x1_err", "c_err"]
dfc = df.dropna(subset=need).copy()

# Ensure covariance columns exist; fill missing with 0
for col in ["cov_x0_x1", "cov_x0_c", "cov_x1_c"]:
    if col not in dfc.columns:
        dfc[col] = 0.0
dfc[["cov_x0_x1","cov_x0_c","cov_x1_c"]] = dfc[["cov_x0_x1","cov_x0_c","cov_x1_c"]].fillna(0.0)

# Optional sanity: x0 must be > 0 for log10
dfc = dfc[np.isfinite(dfc["x0"]) & (dfc["x0"] > 0)]

# Arrays
z   = dfc["redshift"].to_numpy(float)
x0  = dfc["x0"].to_numpy(float)
x1  = dfc["x1"].to_numpy(float)
c   = dfc["c"].to_numpy(float)
sig_x0 = dfc["x0_err"].to_numpy(float)
sig_x1 = dfc["x1_err"].to_numpy(float)
sig_c  = dfc["c_err"].to_numpy(float)
cov_x0_x1 = dfc["cov_x0_x1"].to_numpy(float)
cov_x0_c  = dfc["cov_x0_c"].to_numpy(float)
cov_x1_c  = dfc["cov_x1_c"].to_numpy(float)

# Cosmology for mu_th (use astropy if available, else simple fallback)
try:
    from astropy.cosmology import FlatLambdaCDM
    cosmo = FlatLambdaCDM(H0=70.0, Om0=0.3)
    mu_th = cosmo.distmod(z).value
except Exception:
    c_kms = 299792.458
    H0 = 70.0
    mu_th = 5.0 * np.log10((c_kms * z / H0) * 1.0e6) - 5.0

# Fit (no peculiar velocity term included)
res = fit_salt2_nuisance(
    z=z, mu_th=mu_th,
    x0=x0, x1=x1, c=c,
    sig_x0=sig_x0, sig_x1=sig_x1, sig_c=sig_c,
    cov_x0_x1=cov_x0_x1, cov_x0_c=cov_x0_c, cov_x1_c=cov_x1_c,
    init_alpha=0.14, init_beta=3.1, init_MB=-19.3,
    init_sigma_int=0.10,
    update_sigma_int_mode="MLE"
)

print(f"N used     = {len(z)}")
print("alpha      =", res["alpha"])
print("beta       =", res["beta"])
print("MB         =", res["MB"])
print("sigma_int  =", res["sigma_int"])
print("chi2_red   =", res["chi2_reduced"])
print("Cov(alpha,beta,MB)=\n", res["cov"])

In [ ]:
# import numpy as np
# import pandas as pd

# LN10 = np.log(10.0)

# # -----------------------------------------------
# # 1. Measurement-variance propagation for mu_obs
# # -----------------------------------------------
# def sigma_mu_meas2(alpha, beta,
#                    x0, x1, c,
#                    sig_x0, sig_x1, sig_c,
#                    cov_x0_x1=None, cov_x0_c=None, cov_x1_c=None):
#     """Propagate SALT2 parameter uncertainties into variance of mu_obs."""
#     N = len(x0)
#     if cov_x0_x1 is None: cov_x0_x1 = np.zeros(N)
#     if cov_x0_c  is None: cov_x0_c  = np.zeros(N)
#     if cov_x1_c  is None: cov_x1_c  = np.zeros(N)

#     dmu_dx0 = -2.5 / (x0 * LN10)
#     dmu_dx1 = +alpha
#     dmu_dc  = -beta

#     var = (dmu_dx0**2) * (sig_x0**2) \
#         + (dmu_dx1**2) * (sig_x1**2) \
#         + (dmu_dc**2)  * (sig_c**2)
#     var += 2.0 * (dmu_dx0 * dmu_dx1) * cov_x0_x1
#     var += 2.0 * (dmu_dx0 * dmu_dc)  * cov_x0_c
#     var += 2.0 * (dmu_dx1 * dmu_dc)  * cov_x1_c
#     return var


# # -----------------------------------------------
# # 2. Weighted least squares for [alpha, beta, MB]
# # -----------------------------------------------
# def wls_theta(X, y, w):
#     """Weighted least-squares solution θ̂ = (XᵀWX)⁻¹ XᵀWy."""
#     WX = X * w[:, None]
#     XtWX = X.T @ WX
#     XtWy = X.T @ (w * y)
#     theta = np.linalg.solve(XtWX, XtWy)
#     cov_theta = np.linalg.inv(XtWX)
#     return theta, cov_theta


# # -----------------------------------------------
# # 3. Fit SALT2 nuisance parameters (no MLE σ_int)
# # -----------------------------------------------
# def fit_salt2_nuisance(
#     z, mu_th,
#     x0, x1, c,
#     sig_x0, sig_x1, sig_c,
#     cov_x0_x1=None, cov_x0_c=None, cov_x1_c=None,
#     init_alpha=0.14, init_beta=3.1, init_MB=-19.3,
#     init_sigma_int=0.10,
#     max_outer_iter=50, tol_theta=1e-8, tol_sigint=1e-6
# ):
#     """Fit (α, β, M_B) using iterative weighted least squares; 
#     update σ_int by forcing reduced χ² ≈ 1. No MLE update."""
#     N = len(x0)
#     if cov_x0_x1 is None: cov_x0_x1 = np.zeros(N)
#     if cov_x0_c  is None: cov_x0_c  = np.zeros(N)
#     if cov_x1_c  is None: cov_x1_c  = np.zeros(N)

#     alpha, beta, MB = float(init_alpha), float(init_beta), float(init_MB)
#     sigma_int = float(init_sigma_int)

#     for _ in range(max_outer_iter):
#         y = mu_th + 2.5 * np.log10(x0)
#         X = np.column_stack([x1, -c, np.ones_like(x1)])

#         var_meas = sigma_mu_meas2(alpha, beta, x0, x1, c,
#                                   sig_x0, sig_x1, sig_c,
#                                   cov_x0_x1, cov_x0_c, cov_x1_c)
#         var_tot = var_meas + sigma_int**2
#         w = 1.0 / var_tot

#         theta, cov_theta = wls_theta(X, y, w)
#         alpha_new, beta_new, MB_new = theta.tolist()

#         r = y - X @ theta

#         # Update σ_int by requiring χ²_red ≈ 1
#         chi2 = float((r**2 * w).sum())
#         dof = max(1, N - 3)
#         target = chi2 / dof
#         sigma_int_new = max(0.0, sigma_int * np.sqrt(target))

#         dtheta = np.max(np.abs([alpha_new - alpha, beta_new - beta, MB_new - MB]))
#         ds = abs(sigma_int_new - sigma_int)
#         alpha, beta, MB, sigma_int = alpha_new, beta_new, MB_new, sigma_int_new
#         if dtheta < tol_theta and ds < tol_sigint:
#             break

#     # Final pass
#     y = mu_th + 2.5 * np.log10(x0)
#     X = np.column_stack([x1, -c, np.ones_like(x1)])
#     var_meas = sigma_mu_meas2(alpha, beta, x0, x1, c,
#                               sig_x0, sig_x1, sig_c,
#                               cov_x0_x1, cov_x0_c, cov_x1_c)
#     var_tot = var_meas + sigma_int**2
#     w = 1.0 / var_tot
#     theta, cov_theta = wls_theta(X, y, w)
#     r = y - X @ theta
#     chi2 = float((r**2 * w).sum())
#     dof = max(1, len(y) - 3)
#     chi2_red = chi2 / dof

#     return dict(
#         alpha=alpha, beta=beta, MB=MB,
#         cov=cov_theta,
#         sigma_int=sigma_int,
#         chi2=chi2, dof=dof, chi2_reduced=chi2_red,
#         residuals=r, weights=w
#     )


# # -----------------------------------------------
# # 4. Load ZTF CSV & run fit
# # -----------------------------------------------
# csv_path = "/Users/pittsburghgraduatestudent/repos/myc21_ztf_mu/ZTF_snia_data.csv"
# df = pd.read_csv(csv_path)
# need = ["redshift", "x0", "x1", "c", "x0_err", "x1_err", "c_err"]
# dfc = df.dropna(subset=need).copy()

# for col in ["cov_x0_x1", "cov_x0_c", "cov_x1_c"]:
#     if col not in dfc.columns:
#         dfc[col] = 0.0
# dfc[["cov_x0_x1","cov_x0_c","cov_x1_c"]] = dfc[["cov_x0_x1","cov_x0_c","cov_x1_c"]].fillna(0.0)

# dfc = dfc[np.isfinite(dfc["x0"]) & (dfc["x0"] > 0)]

# z   = dfc["redshift"].to_numpy(float)
# x0  = dfc["x0"].to_numpy(float)
# x1  = dfc["x1"].to_numpy(float)
# c   = dfc["c"].to_numpy(float)
# sig_x0 = dfc["x0_err"].to_numpy(float)
# sig_x1 = dfc["x1_err"].to_numpy(float)
# sig_c  = dfc["c_err"].to_numpy(float)
# cov_x0_x1 = dfc["cov_x0_x1"].to_numpy(float)
# cov_x0_c  = dfc["cov_x0_c"].to_numpy(float)
# cov_x1_c  = dfc["cov_x1_c"].to_numpy(float)

# try:
#     from astropy.cosmology import FlatLambdaCDM
#     cosmo = FlatLambdaCDM(H0=70.0, Om0=0.3)
#     mu_th = cosmo.distmod(z).value
# except Exception:
#     c_kms = 299792.458
#     H0 = 70.0
#     mu_th = 5.0 * np.log10((c_kms * z / H0) * 1.0e6) - 5.0

# res = fit_salt2_nuisance(
#     z=z, mu_th=mu_th,
#     x0=x0, x1=x1, c=c,
#     sig_x0=sig_x0, sig_x1=sig_x1, sig_c=sig_c,
#     cov_x0_x1=cov_x0_x1, cov_x0_c=cov_x0_c, cov_x1_c=cov_x1_c,
#     init_alpha=0.14, init_beta=3.1, init_MB=-19.3,
#     init_sigma_int=0.10
# )

# print(f"N used     = {len(z)}")
# print("alpha      =", res["alpha"])
# print("beta       =", res["beta"])
# print("MB         =", res["MB"])
# print("sigma_int  =", res["sigma_int"])
# print("chi2_red   =", res["chi2_reduced"])
# print("Cov(alpha,beta,MB)=\n", res["cov"])

N used     = 3576
alpha      = 0.13660689030725104
beta       = 2.971371059612116
MB         = 29.735079621311442
sigma_int  = 0.3106754787846309
chi2_red   = 1.000000037204098
Cov(alpha,beta,MB)=
 [[ 2.07056140e-05 -1.75625459e-05 -1.86834495e-07]
 [-1.75625459e-05  1.02295035e-03  4.23263809e-05]
 [-1.86834495e-07  4.23263809e-05  3.27145187e-05]]
